In [53]:
import sqlite3
import langchain
import langchain_community
from langchain_community.utilities import SQLDatabase
import os

In [54]:
#   db_path = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
db_path = r"./chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [55]:
# Pfad zur Datenbank und des aktuellen Arbeitsverzeichnisses checken für Debugging
print(os.path.abspath(db_path))
print(os.getcwd())

c:\Users\flemm\OneDrive - NORDAKADEMIE\01_Master MADS23a\14_MASTERARBEIT\LangChain_POC\LangChain_Lernen\POC-LangChain\chinook-database-master\ChinookDatabase\DataSources\Chinook_Sqlite.sqlite
c:\Users\flemm\OneDrive - NORDAKADEMIE\01_Master MADS23a\14_MASTERARBEIT\LangChain_POC\LangChain_Lernen\POC-LangChain


In [56]:
from Flemmings_Functions.ollama import get_ollama_model_names

In [57]:
model_names = get_ollama_model_names(debug=False)

In [58]:
db_dialect = db.dialect

In [59]:
model_name = model_names[0] # Modell festlegen
model_name

'llama3.2-vision:11b'

In [60]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=model_name,
    temperature=0,
)
print(f'{model_name} ausgewählt und geladen.')

llama3.2-vision:11b ausgewählt und geladen.


In [61]:
from Flemmings_Functions.extract import extract_schema


In [62]:
schema = extract_schema(db_path, sample_rows=1, verbose=False)

In [63]:
schema_info_str = str(schema)

Prompt-Template erstellen

In [64]:
from langchain import PromptTemplate

# Define the prompt template in English
prompt_template = PromptTemplate(
    input_variables=["schema_info", "question","db_dialect"],
    template="""
You are a helpful SQL assistant that provides only SELECT-Statements.
Based on the following database schema, translate a natural language query into an SQL query that is executable in my {db_dialect}-Database.

Database Schema:
{schema_info}

Using the schema information above, please formulate the appropriate SQL query this question:
Question: {question}

SQL Query:
""".strip()
)

In [65]:
question = "In which city is the customer with the highest customer ID located?"

In [66]:
prompt = prompt_template.format(
    schema_info=schema_info_str, 
    question=question, 
    db_dialect=db_dialect
    )


Ab hier mit ChatGPT

In [67]:
from langchain.chains import LLMChain
from typing_extensions import TypedDict
import re

# Define your State class
class State(TypedDict):
    schema_info: str       # Loaded information about the DB schema
    question: str          # Original natural language question
    db_dialect: str        # SQL dialect (e.g., "sqlite")
    parsed_question: str   # (Optional) Parsed or analyzed question
    query: str             # Generated SQL query
    execution_result: str  # Result from executing the SQL query
    answer: str            # Final answer formatted for the user
    error: str             # Error message (if any)

In [68]:
# Function to extract SQL statement from text
def extract_sql_statement(text: str) -> str:
    """
    Extracts the SQL statement from a given text, starting from "SELECT" 
    until the first semicolon ";".
    """
    pattern = r'(SELECT.*?;)'
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""

In [69]:
chain = LLMChain(llm=llm, prompt=prompt_template)

In [71]:
output = chain.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "db_dialect": state["db_dialect"]
        })

NameError: name 'state' is not defined

In [ ]:
# New helper function that uses chain.invoke and updates the State
def extract_sql_query_state(prompt_template, state: State) -> State:
    """
    Uses an LLMChain to generate an SQL query from the natural language question 
    contained in the provided state. Updates the state's 'query' field with the 
    extracted SQL and sets an 'error' field if exceptions occur.
    """
    chain = LLMChain(llm=llm, prompt=prompt_template)
    
    try:
        # Call chain.invoke with a dictionary as input
        output = chain.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "db_dialect": state["db_dialect"]
        })
        
        # Optionally extract the SQL statement (if extra formatting is added)
        sql_query = extract_sql_statement(output)
        state["query"] = sql_query if sql_query else output
        state["error"] = ""
    except Exception as e:
        state["error"] = str(e)
    
    return state

# Example usage:

# Assuming you have created the prompt_template and have the following variables defined:
# schema_info_str, question, and db_dialect


In [ ]:
# Create an initial state
state: State = {
    "schema_info": schema_info_str,
    "question": question,
    "db_dialect": db_dialect,
    "parsed_question": "",
    "query": "",
    "execution_result": "",
    "answer": "",
    "error": ""
}

In [ ]:
# Invoke the chain to update the state with the generated SQL query
state = extract_sql_query_state(prompt_template, state)

print("Generated SQL Query:")
print(state["query"])
if state["error"]:
    print("Error encountered:", state["error"])


Generated SQL Query:

Error encountered: expected string or bytes-like object, got 'dict'


In [ ]:
# Wenn output ein Dictionary ist und den Schlüssel "text" enthält, verwende dessen Inhalt
if isinstance(output, dict) and "text" in output:
    sql_query = extract_sql_statement(output["text"])
else:
    sql_query = extract_sql_statement(output)


In [ ]:
def extract_sql_query_state(prompt_template, state: State) -> State:
    chain = LLMChain(llm=llm, prompt=prompt_template)
    
    try:
        output = chain.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "db_dialect": state["db_dialect"]
        })
        
        # Extrahiere den SQL-Text aus dem Dictionary, falls vorhanden
        if isinstance(output, dict) and "text" in output:
            sql_query = extract_sql_statement(output["text"])
        else:
            sql_query = extract_sql_statement(output)
        
        state["query"] = sql_query if sql_query else output
        state["error"] = ""
    except Exception as e:
        state["error"] = str(e)
    
    return state


In [ ]:
extract_sql_query_state(prompt_template, state)

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe